# Rate Changes (Easy)
This task will set up a simple network where the Poisson rates are updated during the simulation.

Setup a function to be executed at start up, accepting a label and a connection. Using sleep, update the rates of 100 neurons in the Poisson source using the label and the connection at intervals of 0.2 seconds to progressive values of 5Hz, 10Hz, 20Hz and 40Hz.

In [ ]:
from time import sleep

def update_rates(label, connection):
    for rate in [5, 10, 20, 40]:
        sleep(0.2)
        connection.set_rates(label, [(i, rate) for i in range(100)])

Set up a SpynnakerPoissonControlConnection with a poisson label of ```source``` and local port of None.

In [ ]:
import spyNNaker.pyNN as sim

connection = sim.external_devices.SpynnakerPoissonControlConnection(poisson_labels=["source"], local_port=None)

Register the function created previously to be called for the label ```source``` when the simulation starts.

In [ ]:
connection.add_start_resume_callback("source", update_rates)

Setup the simulation with a timestep of 1.0ms.

In [ ]:
sim.setup(1.0)

Create a Population of 100 Poisson neurons with an initial rate of 0Hz, but with the ability to have a rate of up to 40Hz.  Ensure this uses the label ```source```.

In [ ]:
source = sim.Population(100, sim.SpikeSourcePoisson(rate=0), additional_parameters={"max_rate": 40}, label="source")

Create a Population of 100 LIF neurons and record the spikes.

In [ ]:
pop = sim.Population(100, sim.IF_curr_exp(), label="pop")
pop.record("spikes")

Connect the LIF neurons to the Poisson neurons with a fixed probability of 0.5, a weight of 0.5nA and a delay of 1ms.

In [ ]:
sim.Projection(source, pop, sim.FixedProbabilityConnector(0.1), sim.StaticSynapse(weight=5, delay=1))

Add live rate control to the Poisson sources.  Remember to register the connection.

In [ ]:
sim.external_devices.add_poisson_live_rate_control(source, database_notify_port_num=connection.local_port)

Run the simulation for 1000ms, extract the spikes and end the simulation.

In [ ]:
sim.run(1000)
spikes = pop.get_data("spikes").segments[0].spiketrains
sim.end()

Plot the spikes.

In [ ]:
import pyNN.utility.plotting as plot

plot.Figure(
    plot.Panel(spikes, yticks=True, markersize=5, xlim=(0, 1000)),
    title="Spikes"
)